# Desafio Cientista de Dados - Análise Exploratória e Machine Learning


-  ### Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import datetime as dt
from sklearn import linear_model
import pickle

- ### Tipagem e leitura do arquivo .csv

In [ ]:
arquivo = r'C:\Users\Lucas\OneDrive\Documentos\Ligthhouse\data\teste_indicium_precificacao.csv'
tipos = {'id':'int', 'nome':'str', 'host_id':'int', 'host_name':'str', 'bairro_group':'str', 'bairro':'str', 'latitude':'float', 'longitude':'float', 'room_type':'str', 'price': 'float',
         'minimo_noites':'int', 'numero_de_reviews':'float', 'reviews_por_mes':'float', 'calculado_host_listing_count':'int', 'disponibilidade_365':'int'}

df = pd.read_csv(arquivo, sep=',',dtype=tipos, parse_dates=['ultima_review'])

- ### Lendo as oito primeiras linhas do arquivo

In [ ]:
df.head(8)

- ### Lendo últimas linhas de um arquivo

In [ ]:
df.tail(8)

- ### Descobrindo o tamanho da base a qual estamos tratando

In [ ]:
df.shape

- ### Descobrindo os dtypes das colunas

In [ ]:
df.dtypes

- ### Tratando valores nulos

In [ ]:
df.isnull().sum()
df.dropna(inplace=True)

- ### Análise Temporal

In [ ]:
comeco = df['ultima_review'].min()
fim = df['ultima_review'].max()
print(f'O período vai de {comeco} até {fim}')

- ### Tratando dados duplicados

In [ ]:
print(df.duplicated().sum())  

- ### Dados estatísticos do Dataframe:

In [ ]:
df.describe()

- ### Tipo de Imóvel com mais interação

In [ ]:
df_agrupado = df.groupby(['room_type']).agg({'reviews_por_mes': np.mean})
df_agrupado

- ### Preço do aluguel por tipo de imóvel ao longo do tempo

In [ ]:
df_agrupado_por_tipo_de_imovel = df.groupby('room_type', sort=False).agg({'price':np.sum}).reset_index()
px.area(df_agrupado_por_tipo_de_imovel, x='room_type', y ='price')

- ### Analisando a correlação entre o preço de aluguel e número de reviews

In [ ]:
df_correlacao_aluguel_review = df[['price', 'numero_de_reviews']]
df_correlacao_aluguel_review.corr()
plt.rcParams['figure.figsize'] = (18, 9)
ax = sns.heatmap(df_correlacao_aluguel_review.corr(), annot=True)

- ### Os bairros de Manhattan possui os preços de aluguéis mais altos da cidade de New York

#### Supondo que uma pessoa esteja pensando em investir em um apartamento para alugar na plataforma, onde seria mais indicada a compra?


In [ ]:
df_agrupado = df.groupby(['bairro_group', 'room_type']).agg({'price': np.mean})
df_agrupado

### Resposta: Indicaria os apartamentos de Staten Island, por possuírem os menores preços de aluguéis da cidade

In [ ]:
px.histogram(df, x = 'bairro_group', y = 'price')

- ### Correlação entre o número mínimo de noites e a disponibilidade ao longo do ano correlativas com os preços dos aluguéis

#### O número mínimo de noites e a disponibilidade ao longo do ano interferem no preço?


In [ ]:
df_2 = df[['minimo_noites', 'disponibilidade_365', 'price']] 
df_2.corr()
plt.rcParams['figure.figsize'] = (18, 9)
ax = sns.heatmap(df_2.corr(), annot=True)

#### Resposta: Não, a correlação dessas variáveis é um tipo de correlação fraca, pois o seu coeficiente é distante de 1 ou -1

- ### Criando a Regressão Linear

In [ ]:
previsao_precos = linear_model.LinearRegression(fit_intercept=True)

- ### Defino os meus eixos x e y

In [ ]:
teste_x = df['ultima_review'].values.reshape(-1, 1)
teste_y = df['price'].values.reshape(-1, 1)

type(teste_x)
type(teste_y)

- ### Treinando modelo de Machine Learning

In [ ]:
previsao_precos.fit(teste_x, teste_y)

- ### Coeficientes

In [ ]:
print(previsao_precos.coef_)
print(previsao_precos.intercept_)

- ### Modelo de previsão de preços

### Explique como você faria a previsão do preço a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

In [ ]:
modelo_decisao = previsao_precos.predict(teste_y)
modelo_decisao

### Resposta: A previsão de preços seria realizada atráves de um modelo Machine Learning, utilizando a biblioteca sklearn, treinaria o modelo, testaria o modelo e aplicaria para o meu Dataframe, já que estamos envolvendo um problema de regressão, a vantagem é se tratar de um modelo extremamente útil quanto a capacidade de analisar o comportamento dos dados, o contra é a quanto a manutenção nos dados. 

#### Qual seria a sugestão de preço?

### Resposta: Com base no modelo de previsão, a sugestão de preço seria: 285.16 


- ### Gerando .pkl do modelo Machine Learning

In [ ]:
with open(r'C:\Users\Lucas\OneDrive\Documentos\Ligthhouse\machine_learning.pkl', 'wb') as arquivo:
    pickle.dump(modelo_decisao, arquivo)